In [ ]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [ ]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051


In [ ]:
import os
from pyngrok import ngrok

In [ ]:
ngrok.kill()

In [ ]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://unnecessitated-kendall-gogetting.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://unnecessitated-kendall-gogetting.ngrok-free.dev


True

In [ ]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [ ]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [ ]:
result = stateful_query("簡介明新科技大學")
print(result)

明新科技大學（Minghsin University of Science and Technology，簡稱明新科大）是一所位於台灣新竹縣新豐鄉的私立科技大學。學校的創立可追溯至1966年3月，當時以「明新工業專科學校」立案，並於同年3月1日正式招收新生，設有機械、土木、工業管理三科五年制課程。

學校的發展歷程如下：
*   1966年：明新工業專科學校創校。
*   1993年：更名為明新工商專科學校。
*   1997年7月：奉教育部核准改制為「明新技術學院」，並附設專科部。
*   2002年8月：奉教育部核准優先升格為「明新科技大學」，並沿用此校名。
*   2018年12月：奉教育部核准更名為「明新學校財團法人明新科技大學」。

明新科技大學的校名「明新」取自《大學》「在明明德，在新民，在止於至善」的精義，旨在闡揚人類與生俱來的德性與情操，培育學子具備高尚品德、專業學問與優良技術，以達成「一流產業大學」的發展願景，並培育出「跨域整合、務實創新、全人學習」的專業人才，符合校訓「堅毅、求新、創造」的精神。

目前，明新科技大學設有六個學院，包括半導體學院、工程學院、管理學院、民生學院、人文與設計學院以及共同教育學院。學校涵括20個學系、2個學位學程（含1個博士學位學程）以及11個碩士班。截至目前，學生總數（含日間部與進修部）約有一萬三千餘人，專任教師約三百八十餘人，畢業校友已逾九萬餘人。2022年10月，學校獲教育部核准通過「半導體科技博士學位學程」，這是明新科大成立五十六年來的第一個博士班。


In [ ]:
result2 = stateful_query("校長是誰？")
print(result2)

明新科技大學現任校長是**呂明峯**教授。他於2025年2月1日正式上任，成為明新科大第11任校長。

呂明峯教授在接任校長一職前，曾擔任明新科技大學電子工程系教授、產學合作處處長、工學院院長、半導體學院院長等職務，擁有豐富的半導體教育及產業實務經驗。 他將以「四大核心模組」作為校務治理策略，帶領明新科大開創技職教育新局，目標是將明新科大打造成為「具國際魅力的產業科技大學」。

在呂明峯校長上任之前，明新科技大學的校長是林啟瑞。


In [ ]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"file","id":"592603361938506394","markAsReadToken":"k6Mcp60CAZd367o74TujGgVRFAZW2IUDhrBg-VXdmRQJpx3UUz9ENfLhgjiD2OOFynlzJ-EaaITolYNUJJTj0AvPo3MlCcnqEf_mVe-B9EzROL_nNh9MncCrALLGpUqGyXIE9oJHQDYrBTWapxhT9IdjvZIm9DAwGvHR27RlZRFnd2X-Wqmz-T4C8Ku3K9WwvwczYngj1uDkmvsPW7iTGw","fileName":"faq.txt","fileSize":27,"contentProvider":{"type":"line"}},"webhookEventId":"01KCRCY2EZFFHPG9Y2Y2N1Q95T","deliveryContext":{"isRedelivery":false},"timestamp":1766050433024,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"278afb88bf1149b48e81f9ca0e26f905","mode":"active"}]}


BODY:  {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"file","id":"592603361938506394","markAsReadToken":"k6Mcp60CAZd367o74TujGgVRFAZW2IUDhrBg-VXdmRQJpx3UUz9ENfLhgjiD2OOFynlzJ-EaaITolYNUJJTj0AvPo3MlCcnqEf_mVe-B9EzROL_nNh9MncCrALLGpUqGyXIE9oJHQDYrBTWapxhT9IdjvZIm9DAwGvHR27RlZRFnd2X-Wqmz-T4C8Ku3K9WwvwczYngj1uDkmvsPW7iTGw","fileName":"faq.txt","fileSize":27,"contentProvider":{"type":"line"}},"webhookEventId":"01KCRCY2EZFFHPG9Y2Y2N1Q95T","deliveryContext":{"isRedelivery":false},"timestamp":1766050433024,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"278afb88bf1149b48e81f9ca0e26f905","mode":"active"}]}
檔案已下載：/content/uploaded_files/faq.txt
檔案已上傳到 Gemini：https://generativelanguage.googleapis.com/v1beta/files/oi69zjip7is6


INFO:werkzeug:127.0.0.1 - - [18/Dec/2025 09:33:55] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"text","id":"592603383681515949","quoteToken":"KBQIWXL9gm3dzV1nuedqteIzX6JoyyclAFiXXif8uDQ1sMZpYuURsdugmEEUIA5vm2w73RUiCA9H5YEG6pTYEvE2T3X2a30oJcOQib8ZFG1HOQX2uDDUrFhbKYftD5khDjj88ZVKcJExgyIi2b1ElA","markAsReadToken":"fjdPXIFOV340xQtHOpZb4ihsBZfcaF3adQS8uAjZ1VxEbIzwq3n7SIQLskLXnTfNIxLofPr4uRxsrk18jhJhyGPxJUhNl-kvQUpt3aYgF2BXQ-6SndFGRE7FjO2d-UdJ6_cPj-9awO98xPNi-LXkjW3sINS8tH4I5CftE4oqfFopfBTVamE3yIkZFpn6CMjr7LjA6eobS-lVjZc0gv2zpg","text":"AI 大俠愛吃甚麼?"},"webhookEventId":"01KCRCYEXX2C4GYHQ77TEQ92GT","deliveryContext":{"isRedelivery":false},"timestamp":1766050445966,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"07870c2afccc46dfa9ca6aff19926044","mode":"active"}]}


BODY:  {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"text","id":"592603383681515949","quoteToken":"KBQIWXL9gm3dzV1nuedqteIzX6JoyyclAFiXXif8uDQ1sMZpYuURsdugmEEUIA5vm2w73RUiCA9H5YEG6pTYEvE2T3X2a30oJcOQib8ZFG1HOQX2uDDUrFhbKYftD5khDjj88ZVKcJExgyIi2b1ElA","markAsReadToken":"fjdPXIFOV340xQtHOpZb4ihsBZfcaF3adQS8uAjZ1VxEbIzwq3n7SIQLskLXnTfNIxLofPr4uRxsrk18jhJhyGPxJUhNl-kvQUpt3aYgF2BXQ-6SndFGRE7FjO2d-UdJ6_cPj-9awO98xPNi-LXkjW3sINS8tH4I5CftE4oqfFopfBTVamE3yIkZFpn6CMjr7LjA6eobS-lVjZc0gv2zpg","text":"AI 大俠愛吃甚麼?"},"webhookEventId":"01KCRCYEXX2C4GYHQ77TEQ92GT","deliveryContext":{"isRedelivery":false},"timestamp":1766050445966,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"07870c2afccc46dfa9ca6aff19926044","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [18/Dec/2025 09:34:07] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"text","id":"592603401348186301","quoteToken":"u7VOapCiB1fIwYY88H8rzqUjx59aPQBSeN_zb4FR1knEqavRcAkFIjmCZCB9Gvv4u24CdHyYxFmhnjpLpKzXynJKAN28qMST6t8LwFCOZJ_2YegcNSvF0t1MdUB25TpofbWm5uCj0Kh-13L19nNOgw","markAsReadToken":"YQeisFwUvsxhgovsr5K0lRdtUIL5KX6k1NF1qCJaIF1cEkO6eIwxb-r5Uud_yKSq1-yPZ6Ebkuw43Xxrazad0Wiif9Bv09j4qAMHr_sPP0rNg8EtiSzr3pf1b25oU2ArsLVgPy13eLUrPnWVJBkpvIiMmxqCm36pEaFWBuxQecd_4BMlQyNf9JEDHrZyQvRvGB9ay4sW9qGFvOhtrDsUDw","text":"AI 大俠愛吃甚麼?"},"webhookEventId":"01KCRCYS4M1DS5VKW220FZAPCA","deliveryContext":{"isRedelivery":false},"timestamp":1766050456383,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"821f71a25c4f4120892c60b4172d945b","mode":"active"}]}


BODY:  {"destination":"U4266981d0cd4ada66e0c3da0512201fb","events":[{"type":"message","message":{"type":"text","id":"592603401348186301","quoteToken":"u7VOapCiB1fIwYY88H8rzqUjx59aPQBSeN_zb4FR1knEqavRcAkFIjmCZCB9Gvv4u24CdHyYxFmhnjpLpKzXynJKAN28qMST6t8LwFCOZJ_2YegcNSvF0t1MdUB25TpofbWm5uCj0Kh-13L19nNOgw","markAsReadToken":"YQeisFwUvsxhgovsr5K0lRdtUIL5KX6k1NF1qCJaIF1cEkO6eIwxb-r5Uud_yKSq1-yPZ6Ebkuw43Xxrazad0Wiif9Bv09j4qAMHr_sPP0rNg8EtiSzr3pf1b25oU2ArsLVgPy13eLUrPnWVJBkpvIiMmxqCm36pEaFWBuxQecd_4BMlQyNf9JEDHrZyQvRvGB9ay4sW9qGFvOhtrDsUDw","text":"AI 大俠愛吃甚麼?"},"webhookEventId":"01KCRCYS4M1DS5VKW220FZAPCA","deliveryContext":{"isRedelivery":false},"timestamp":1766050456383,"source":{"type":"user","userId":"U0eb3bfa63bf656ad470ba1f81015848e"},"replyToken":"821f71a25c4f4120892c60b4172d945b","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [18/Dec/2025 09:34:21] "POST / HTTP/1.1" 200 -
